# PIN Estimation

In [1]:
import pandas as pd

from fico.modules import dataset
from fico.modules import pin_estimation

## Load Data

In [10]:
quotes = dataset.load_cedro_quotes()
quotes.sample(5)

,symbol,bucket,open,high,low,close,volume,comprador,indefinido,vendedor,naodireto,direto,rlp
22811,LWSA3,2022-12-15,6.98,7.37,6.83,7.11,14480600,8848,351,10015,18962,0,252
9898,WHRL3,2023-07-28,4.98,4.98,4.95,4.95,1600,2,0,5,7,0,0
21162,BBDC4,2023-01-30,13.61,13.79,13.49,13.79,58048100,22822,2047,17411,40777,8,1495
10794,EZTC3,2023-08-17,23.75,23.94,22.88,23.47,2379900,3102,118,4711,7931,0,0
98809,BRFS3,2022-10-27,12.02,12.53,11.94,12.32,14095700,12596,233,10045,22872,2,0


## Estimate PINs

In [11]:
filter = False
if filter:
    stocks = {'PETR4', 'VALE3', 'MGLU3'}
    quotes = quotes[quotes['symbol'].isin(stocks)].copy()

In [12]:
estimate_all_pins = False
if estimate_all_pins:
    pins = pin_estimation.estimate_all_pins(quotes, window=60, verbose=True)
else:
    pins = dataset.load_results()['pin_results']
pins.sample(5)

,symbol,period,pin,alpha,delta,mu,eps_b,eps_s
48341,SANB3,2023-07-05,0.174378,0.300000,0.277775,513.766977,373.150177,356.603704
5381,ELET6,2023-03-14,0.106660,0.350004,0.238115,2914.293649,4173.962291,4369.268063
68348,POSI3,2023-07-10,0.156481,0.333329,0.300018,2694.345450,2267.482634,2573.786460
40066,YDUQ3,2023-02-17,0.145910,0.333321,0.349973,5951.707882,5762.672987,5849.725071
58733,GFSA3,2023-08-25,0.204987,0.199993,0.166677,4236.793757,1366.658524,1919.582691


# Portfolio Build

In [13]:
from fico.modules import stock_selection
from fico.modules import portfolio_build

In [14]:
eco_data = dataset.load_economatica_quotes()
print(f'Eco-Data type is {type(eco_data)}. Its keys are {eco_data.keys()}')

Eco-Data type is <class 'dict'>. Its keys are dict_keys(['volumes', 'ibov', 'closing_prices'])


In [16]:
eligible_stocks = stock_selection.filter(eco_data['volumes'])
portfolios = portfolio_build.build_portfolio(
    pins,
    eligible_stocks,
)
portfolios

{'pins': {'long':        symbol     period       pin     alpha     delta            mu  \
  4018    SEQL3 2022-06-30  0.193954  0.433354  1.000000   2281.942025   
  57191  AURA33 2022-07-29  0.228066  0.333326  0.699991   3422.926164   
  13534   JHSF3 2022-08-31  0.178792  0.449996  0.777770   2332.391765   
  8483    CASH3 2022-09-30  0.236780  0.083325  0.799966  37872.301500   
  22143   OPCT3 2022-09-30  0.220967  0.416672  0.760001    867.807652   
  ...       ...        ...       ...       ...       ...           ...   
  47735   RCSL3 2023-08-28  0.184377  0.349998  0.714287   1007.422720   
  58833   GFSA3 2023-08-28  0.169418  0.083328  1.000000   9013.758068   
  61627   COCE5 2023-08-28  0.209680  0.350504  0.945526     54.906982   
  76149   SEER3 2023-08-28  0.174740  0.333330  0.750000   1454.902737   
  77004   RAIZ4 2023-08-28  0.273132  0.533351  0.781247   9270.317291   
  
               eps_b        eps_s  
  4018   2328.535814  1781.142529  
  57191  2059.777061 

## Calculate Returns

In [8]:
from fico.modules import returns

In [9]:
r = returns.calculate_all_portfolios_returns(portfolios['weights'], eco_data['closing_prices'])
r

,long,short,longonly,long_short
2022-07-01,-0.048870,0.003752,-0.049360,-0.052622
2022-07-04,-0.039894,-0.018692,-0.040385,-0.021203
2022-07-05,0.042575,0.011429,0.042084,0.031146
2022-07-06,0.000490,0.008475,0.000000,-0.007984
2022-07-07,0.004337,0.010271,0.003846,-0.005934
...,...,...,...,...
2023-09-01,0.014920,0.026360,0.014429,-0.011441
2023-09-04,-0.005777,-0.018228,-0.006267,0.012452
2023-09-05,-0.011019,0.029056,-0.011509,-0.040075
2023-09-06,-0.004292,-0.015351,-0.004783,0.011059
